In [49]:
!ls
import numpy as np

Ge                  Ge_z_fc1_fc_b:0.npy Se_y_fc1_fc_w:0.npy feature_std_C.npy
GeSe2.xyz           Ge_z_fc1_fc_w:0.npy Se_y_fc2_fc_b:0.npy feature_std_Si.npy
Ge_x_fc1_fc_b:0.npy Ge_z_fc2_fc_b:0.npy Se_y_fc2_fc_w:0.npy force_mean_C.npy
Ge_x_fc1_fc_w:0.npy Ge_z_fc2_fc_w:0.npy Se_y_fc3_fc_b:0.npy force_mean_Si.npy
Ge_x_fc2_fc_b:0.npy Ge_z_fc3_fc_b:0.npy Se_y_fc3_fc_w:0.npy force_std_C.npy
Ge_x_fc2_fc_w:0.npy Ge_z_fc3_fc_w:0.npy Se_y_fc4_fc_b:0.npy force_std_Si.npy
Ge_x_fc3_fc_b:0.npy Ge_z_fc4_fc_b:0.npy Se_y_fc4_fc_w:0.npy get_force.ipynb
Ge_x_fc3_fc_w:0.npy Ge_z_fc4_fc_w:0.npy Se_z_fc1_fc_b:0.npy input.xyz
Ge_x_fc4_fc_b:0.npy Se                  Se_z_fc1_fc_w:0.npy input_Gese2.xyz
Ge_x_fc4_fc_w:0.npy Se_x_fc1_fc_b:0.npy Se_z_fc2_fc_b:0.npy model_detain.txt
Ge_y_fc1_fc_b:0.npy Se_x_fc1_fc_w:0.npy Se_z_fc2_fc_w:0.npy predicted_force.xyz
Ge_y_fc1_fc_w:0.npy Se_x_fc2_fc_b:0.npy Se_z_fc3_fc_b:0.npy train_XX.npy

Ge_y_fc2_fc_w:0.npy Se_x_fc3_fc_b:0.npy Se_z_fc4_fc_b:0.npy train_atype.npy
Ge_y_f

In [76]:
with open('./predicted_force.xyz','r') as fin:
    data = fin.readline()
    num_atoms = int(data)
    data = fin.readline()
    force = []; position = []
    for line in fin:
        data = line.split()
        position.append([float(data[1]),float(data[2]),float(data[3])])
        force.append([float(data[4]),float(data[5]),float(data[6])])

xx = np.load('./train_XX.npy')
yy = np.load('./train_YY.npy')

print('xx.shape: ', xx.shape, ' yy.shape: ', yy.shape)
for i in range(384):
    print(i,position[i][:],'\t',force[i][:],'\t',yy[i][:])


xx.shape:  (24576, 930)  yy.shape:  (24576, 3)
0 [5.26614, 5.11941, 11.03744] 	 [-0.82957, 0.39822, 0.27814] 	 [-0.900056  0.370832  0.274648]
1 [5.00188, 5.7447, 1.75911] 	 [-1.63422, -0.63682, 0.34415] 	 [-1.758569 -0.652092  0.43582 ]
2 [1.26252, 11.77625, 8.05721] 	 [0.20346, 0.3299, 0.07085] 	 [0.187129 0.383461 0.081279]
3 [6.61541, 11.09954, 17.97799] 	 [-0.15357, 0.80142, -0.58404] 	 [-0.191456  0.852271 -0.62424 ]
4 [4.27262, 14.98928, 10.08151] 	 [0.09381, -0.0431, -0.50044] 	 [ 0.102951 -0.051976 -0.509993]
5 [8.30284, 18.76823, 1.1885] 	 [0.34539, -0.29469, -0.5443] 	 [ 0.356611 -0.325859 -0.561332]
6 [9.02707, 20.40483, 9.48392] 	 [0.34687, -0.31182, -0.24131] 	 [ 0.338083 -0.299452 -0.225405]
7 [4.55618, 1.66242, 0.00865] 	 [0.5503, -0.35994, 0.25037] 	 [ 0.575993 -0.333861  0.308235]
8 [8.57243, 5.16092, 11.10061] 	 [0.68528, 0.43537, 0.29408] 	 [0.78222  0.413577 0.218959]
9 [11.88931, 5.27074, 0.66478] 	 [-0.62825, -0.31223, 0.02136] 	 [-0.676877 -0.271041  0.004048]
1

In [133]:
b_x = []
b_x.append(np.load('./Ge_x_fc1_fc_b:0.npy'))
b_x.append(np.load('./Ge_x_fc2_fc_b:0.npy'))
b_x.append(np.load('./Ge_x_fc3_fc_b:0.npy'))
b_x.append(np.load('./Ge_x_fc4_fc_b:0.npy'))

w_x=[]
w_x.append(np.load('./Ge_x_fc1_fc_w:0.npy'))
w_x.append(np.load('./Ge_x_fc2_fc_w:0.npy'))
w_x.append(np.load('./Ge_x_fc3_fc_w:0.npy'))
w_x.append(np.load('./Ge_x_fc4_fc_w:0.npy'))

mean = np.load('./feature_mean_Si.npy')
stddev = np.load('./feature_std_Si.npy')

nlayers = 4 # total layers
nfeatures = 310  # Number of radial & angular features
mean_x = mean[0:nfeatures] # mean & stddev only for x-direction
stddev_x = stddev[0:nfeatures]

xx = np.load('./train_XX.npy')
iatom=0
x1=xx[iatom][0:nfeatures]

print('xx&x1&mean&stddev: ', xx.shape,x1.shape,mean_x.shape, stddev_x.shape)

# correct the feature values by mean & stddev
xnew = (x1 - mean_x)/stddev_x

for layer in range(nlayers):
    xnew = xnew.dot(w_x[layer])+b_x[layer]
    
    # relu except the last step
    if layer < nlayers-1:
        xnew = np.maximum(xnew,0)
    print('layer,xnew.shape: ', layer, xnew.shape, '\t predicted: ', xnew[0], 
          '\tw&b shapes: ', w_x[layer].shape, b_x[layer].shape)

print('\npredicted force: ', xnew) # scaling factor, 25x?

xx&x1&mean&stddev:  (24576, 930) (310,) (310,) (310,)
layer,xnew.shape:  0 (512,) 	 predicted:  0.0 	w&b shapes:  (310, 512) (512,)
layer,xnew.shape:  1 (1024,) 	 predicted:  0.3070735 	w&b shapes:  (512, 1024) (1024,)
layer,xnew.shape:  2 (512,) 	 predicted:  0.33473638 	w&b shapes:  (1024, 512) (512,)
layer,xnew.shape:  3 (1,) 	 predicted:  -10.45724 	w&b shapes:  (512, 1) (1,)

predicted force:  [-10.45724]
